<a href="https://colab.research.google.com/github/erinijapranckeviciene/MF54609_18981_1_20241/blob/main/Copy_of_FC_Chapter11_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### BTW, Recalling the effect of validation error being less than training

https://pyimagesearch.com/2019/10/14/why-is-my-validation-loss-lower-than-my-training-loss/#:~:text=Regularization%20methods%20often%20sacrifice%20training,lower%20than%20your%20training%20loss.

### Prepare IMDB data as before

In [5]:
!wget https://github.com/erinijapranckeviciene/MF54609_18981_1_20241/raw/refs/heads/main/datasets/RNN/aclImdb.zip
!unzip -qq aclImdb.zip

--2025-01-17 22:04:30--  https://github.com/erinijapranckeviciene/MF54609_18981_1_20241/raw/refs/heads/main/datasets/RNN/aclImdb.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/erinijapranckeviciene/MF54609_18981_1_20241/refs/heads/main/datasets/RNN/aclImdb.zip [following]
--2025-01-17 22:04:31--  https://raw.githubusercontent.com/erinijapranckeviciene/MF54609_18981_1_20241/refs/heads/main/datasets/RNN/aclImdb.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 73380778 (70M) [application/zip]
Saving to: ‘aclImdb.zip’

aclImdb.zip         100%[===================>]  69.98M   172MB/s    in 0.

In [6]:
import os, pathlib, shutil, random
from tensorflow import keras

batch_size = 32
train_ds = keras.utils.text_dataset_from_directory("aclImdb/train", batch_size=batch_size )
val_ds = keras.utils.text_dataset_from_directory("aclImdb/val", batch_size=batch_size)
test_ds = keras.utils.text_dataset_from_directory("aclImdb/test", batch_size=batch_size)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


### text only ds

In [7]:
# This function returns only data part without target
# to create a new dataset that will be used to create dictionary
text_only_train_ds = train_ds.map(lambda x, y: x)
for inputs in text_only_train_ds:
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("inputs[0]:", inputs[0])
    break

inputs.shape: (32,)
inputs.dtype: <dtype: 'string'>
inputs[0]: tf.Tensor(b'- The movie opens with a meteor crashing into a lake. Unbeknownst to the locals, a dinosaur egg is also at the bottom of the lake. The meteor heats the lake, turning it into a giant incubator. You guessed it, the egg hatches releasing a dinosaur that proceeds to terrorize the community.<br /><br />- What utter garbage. It\'s not that I mind the stop-motion clay dinosaur, it\'s everything else about the movie that bothers me. The acting is atrocious. The dialogue is utterly ridiculous. The comic relief is anything but comic. Logic is non-existent. Any similarity between the "scientists" in this movie and an actual scientist is purely coincidental. I could go on for an eternity on the bad aspects of this movie, but you get the idea. I feel it\'s fairly safe to call this disaster "MST3K Worthy".', shape=(), dtype=string)


### Prepare int representation dataset

In [8]:
import tensorflow as tf
from tensorflow.keras import layers

# max_length is a length of the vector that encodes text
# 250 size gave ~0.85 on test data
# 600 with 4 LSTM units does not train at all
# the reviews are about 300 words
max_length = 300
max_tokens = 20000
text_vectorization = layers.TextVectorization( max_tokens=max_tokens, output_mode="int", output_sequence_length=max_length)

text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
int_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
int_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)

### Verify

In [9]:
for inputs, targets in int_train_ds:
    print(inputs.shape)
    print(inputs[0])
    print(targets[0])
    # With this test_input variable verify tf.one_hot() transformation
    test_input=inputs[0]
    break

(32, 300)
tf.Tensor(
[   17     4   420    39    12    30   742     3   705    57 12064    86
    11    18    42   117    22  1236     6    70  2564   215     4   164
 13607  2830   379     3     4   110   616    19     2    93     8    63
     2    18    14   318     3     2   427   738    42  3447    70    15
    34  3090     3  2946   574    31     1  1470  6737     2  5859   116
   496    17     4   230  9575     3  4877     4   243  6414    31     2
   502     5     2  1364    28     3    25     1   592    24  8927    58
   351     2   592  1905    46     3 17524     6   884    25  1048    46
     8     2  1281    28   602    58  5481    40    78     4   310   112
    35  5751     3    28   299   261    40     4  1568     5  3375   448
     8     2  3390     2  2584  9986 19732  6255     5    40   165  1460
    36  3584    40     4     1  3479     3    40  3185 17565  1369    28
  2300     6    40    12    28     7   166     6   504    96    32    55
  4172    58   856    90     3

## Transformers  F.Chollet Chapter 11

#### 11.4 The Transformer architecture

..."Transformers were introduced in the seminal paper “Attention is all you need” by Vaswani et al. The gist of the paper is right there in the title: as it turned out, a simple mechanism called “neural attention” could be used to build powerful sequence models that didn’t feature any recurrent layers or convolution layers."...

### Listing 11.21 Transformer encoder implemented as a subclassed Layer

In [10]:
import tensorflow as tf
from tensorflow import keras
from keras import layers

class TransformerEncoder(layers.Layer):
  def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
    super().__init__(**kwargs)
    self.embed_dim = embed_dim
    self.dense_dim = dense_dim
    self.num_heads = num_heads
    self.attention = layers.MultiHeadAttention(
        num_heads=num_heads, key_dim=embed_dim)
    self.dense_proj = keras.Sequential(
        [layers.Dense(dense_dim, activation="relu"), layers.Dense(embed_dim),]
    )
    self.layernorm_1 = layers.LayerNormalization()
    self.layernorm_2 = layers.LayerNormalization()

  def call(self, inputs, mask=None):
    if mask is not None:
        mask=mask[:, tf.newaxis, :]
    attention_output = self.attention(
        inputs, inputs, attention_mask=mask
    )
    proj_input = self.layernorm_1(inputs + attention_output)
    proj_output = self.dense_proj(proj_input)
    return self.layernorm_2(proj_input + proj_output)

  def get_config(self):
    config = super().get_config()
    config.update({
        "embed_dim": self.embed_dim,
        "num_heads": self.num_heads,
        "dense_dim": self.dense_dim,
    })
    return config


### Listing 11.22 Using the Transformer encoder for text classification

In [11]:
vocab_size = 20000
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None,), dtype="int64")
x = layers.Embedding(vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)

model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, None)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, None, 256)           │       5,120,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_encoder                  │ (None, None, 256)           │         543,776 │
│ (TransformerEncoder)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_max_pooling1d                 │ (None, 256)                 │               0 │
│ (GlobalMaxPooling1D)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │             257 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,664,033 (21.61 MB)

 Trainable params: 5,664,033 (21.61 MB)

 Non-trainable params: 0 (0.00 B)

### Listing 11.23 Training and evaluating the Transformer encoder based model

In [12]:
callbacks = [ keras.callbacks.ModelCheckpoint("transformer_encoder.keras", save_best_only=True) ]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=20, callbacks=callbacks)

model = keras.models.load_model("transformer_encoder.keras", custom_objects={"TransformerEncoder": TransformerEncoder})
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")


Epoch 1/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 34s 36ms/step - accuracy: 0.6548 - loss: 0.7535 - val_accuracy: 0.8264 - val_loss: 0.3882
Epoch 2/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 20s 33ms/step - accuracy: 0.8204 - loss: 0.3955 - val_accuracy: 0.8508 - val_loss: 0.3515
Epoch 3/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 21s 33ms/step - accuracy: 0.8470 - loss: 0.3534 - val_accuracy: 0.8426 - val_loss: 0.3639
Epoch 4/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 41s 34ms/step - accuracy: 0.8670 - loss: 0.3198 - val_accuracy: 0.8612 - val_loss: 0.3230
Epoch 5/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 21s 33ms/step - accuracy: 0.8685 - loss: 0.3043 - val_accuracy: 0.8634 - val_loss: 0.3271
Epoch 6/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 41s 34ms/step - accuracy: 0.8886 - loss: 0.2690 - val_accuracy: 0.8630 - val_loss: 0.3282
Epoch 7/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 21s 34ms/step - accuracy: 0.8971 - loss: 0.2533 - val_accuracy: 0.8664 - val_loss: 0.3375
Epoch 8/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 41s 34ms/step - accuracy: 0.9077 - loss: 0.2285 - 

/usr/local/lib/python3.11/dist-packages/keras/src/layers/layer.py:391: UserWarning: `build()` was called on layer 'transformer_encoder', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


782/782 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.8580 - loss: 0.3210
Test acc: 0.859


### Listing 11.24 Implementing positional embedding as a subclassed layer

In [13]:
class PositionalEmbedding(layers.Layer):
  def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
    super().__init__(**kwargs)
    self.token_embeddings = layers.Embedding(
        input_dim=input_dim, output_dim=output_dim)
    self.position_embeddings = layers.Embedding(
        input_dim=sequence_length, output_dim=output_dim)
    self.sequence_length = sequence_length
    self.input_dim = input_dim
    self.output_dim = output_dim

  def call(self, inputs):
    length = tf.shape(inputs)[-1]
    positions = tf.range(start=0, limit=length, delta=1)
    embedded_tokens = self.token_embeddings(inputs)
    embedded_positions = self.position_embeddings(positions)
    return embedded_tokens + embedded_positions

  #def compute_mask(self, inputs, mask=None):
  #  return tf.math.not_equal(inputs, 0) #it was a mistake not clear how to fix it
    #return layers.Lambda(lambda x: tf.math.not_equal(x, 0))(inputs)

  def get_config(self):
    config = super().get_config()
    config.update({
      "output_dim": self.output_dim,
      "sequence_length": self.sequence_length,
      "input_dim": self.input_dim,
      })
    return config

### Listing 11.25 Combining the Transformer encoder with positional embedding

In [14]:
vocab_size = 20000
sequence_length = 600
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None,), dtype="int64")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)

model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)           │ (None, None)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ positional_embedding                 │ (None, None, 256)           │       5,273,600 │
│ (PositionalEmbedding)                │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_encoder_1                │ (None, None, 256)           │         543,776 │
│ (TransformerEncoder)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_max_pooling1d_1               │ (None, 256)                 │               0 │
│ (GlobalMaxPooling1D)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 1)                   │             257 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,817,633 (22.19 MB)

 Trainable params: 5,817,633 (22.19 MB)

 Non-trainable params: 0 (0.00 B)

### Change the dataset - the sequence length is 600


In [15]:
text_vectorization = layers.TextVectorization( max_tokens=vocab_size, output_mode="int", output_sequence_length=sequence_length)

text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
int_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
int_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)

### Execute the model

In [16]:
callbacks = [keras.callbacks.ModelCheckpoint("full_transformer_encoder.keras", save_best_only=True)]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=20, callbacks=callbacks)



Epoch 1/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 72s 103ms/step - accuracy: 0.6102 - loss: 0.7869 - val_accuracy: 0.8050 - val_loss: 0.4229
Epoch 2/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 73s 98ms/step - accuracy: 0.8006 - loss: 0.4354 - val_accuracy: 0.8288 - val_loss: 0.3810
Epoch 3/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 62s 99ms/step - accuracy: 0.8288 - loss: 0.3786 - val_accuracy: 0.8318 - val_loss: 0.3805
Epoch 4/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 83s 101ms/step - accuracy: 0.8501 - loss: 0.3445 - val_accuracy: 0.8508 - val_loss: 0.3525
Epoch 5/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 83s 102ms/step - accuracy: 0.8676 - loss: 0.3094 - val_accuracy: 0.8518 - val_loss: 0.3473
Epoch 6/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 63s 101ms/step - accuracy: 0.8874 - loss: 0.2726 - val_accuracy: 0.8538 - val_loss: 0.3476
Epoch 7/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 82s 101ms/step - accuracy: 0.9036 - loss: 0.2389 - val_accuracy: 0.8488 - val_loss: 0.3728
Epoch 8/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 63s 100ms/step - accuracy: 0.9222 - loss: 0.2

In [17]:
model = keras.models.load_model("full_transformer_encoder.keras", custom_objects={"TransformerEncoder": TransformerEncoder, "PositionalEmbedding": PositionalEmbedding})
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/layer.py:391: UserWarning: `build()` was called on layer 'positional_embedding', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/layers/layer.py:391: UserWarning: `build()` was called on layer 'transformer_encoder_1', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


782/782 ━━━━━━━━━━━━━━━━━━━━ 15s 18ms/step - accuracy: 0.8447 - loss: 0.3483
Test acc: 0.843


### 11.5.1 A machine translation example

In [18]:
!wget http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zipy
!unzip -q spa-eng.zip

--2025-01-17 22:41:16--  http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.141.207, 74.125.137.207, 142.250.101.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.141.207|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2638744 (2.5M) [application/zip]
Saving to: ‘spa-eng.zip.1’

spa-eng.zip.1       100%[===================>]   2.52M  --.-KB/s    in 0.007s  

2025-01-17 22:41:16 (344 MB/s) - ‘spa-eng.zip.1’ saved [2638744/2638744]

replace spa-eng/_about.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace spa-eng/spa.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y


### Parse the file


In [19]:
text_file = "spa-eng/spa.txt"
with open(text_file) as f:
  lines = f.read().split("\n")[:-1]
text_pairs = []

for line in lines:
  eng, spa = line.split("\t")
  spa = "[start] " + spa + " [end]"
  text_pairs.append((eng, spa))

import random
print(random.choice(text_pairs))


('They were stranded on a deserted island.', '[start] Ellos estaban atrapados en una isla desierta. [end]')


### Shuffle and create the training, validation and test sets

In [20]:
import random
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]

### Listing 11.26 Vectorizing the English and Spanish text pairs

Note that for a non-toy translation model, we would treat punctuation characters as sep-
arate tokens rather than stripping them, since we would want to be able to generate cor-
rectly punctuated sentences. In our case, for simplicity, we’ll get rid of all punctuation.

In [21]:
import tensorflow as tf
import string
import re
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

def custom_standardization(input_string):
  lowercase = tf.strings.lower(input_string)
  return tf.strings.regex_replace(lowercase, f"[{re.escape(strip_chars)}]", "")

#To keep things simple, we’ll only look at
#the top 15,000 words in each language,
#and we’ll restrict sentences to 20 words.

vocab_size = 15000
sequence_length = 20

source_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
target_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)
train_english_texts = [pair[0] for pair in train_pairs]
train_spanish_texts = [pair[1] for pair in train_pairs]
source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_spanish_texts)


### Listing 11.27 Preparing datasets for the translation task

Finally, we can turn our data into a tf.data pipeline. We want it to return a tuple
(inputs, target) where inputs is a dict with two keys, “encoder_inputs” (the English
sentence) and “decoder_inputs” (the Spanish sentence), and target is the Spanish
sentence offset by one step ahead.

In [22]:
batch_size = 64
def format_dataset(eng, spa):
  eng = source_vectorization(eng)
  spa = target_vectorization(spa)
  return ({"english": eng,"spanish": spa[:, :-1],}, spa[:, 1:])

def make_dataset(pairs):
  eng_texts, spa_texts = zip(*pairs)
  eng_texts = list(eng_texts)
  spa_texts = list(spa_texts)
  dataset = tf.data.Dataset.from_tensor_slices((eng_texts, spa_texts))
  dataset = dataset.batch(batch_size)
  dataset = dataset.map(format_dataset, num_parallel_calls=4)
  return dataset.shuffle(2048).prefetch(16).cache()

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)
#


### Dataset outputs


In [23]:
for inputs, targets in train_ds.take(1):
  print(f"inputs['english'].shape: {inputs['english'].shape}")
  print(f"inputs['spanish'].shape: {inputs['spanish'].shape}")
  print(f"targets.shape: {targets.shape}")


inputs['english'].shape: (64, 20)
inputs['spanish'].shape: (64, 20)
targets.shape: (64, 20)


## 11.5.2 Sequence-to-sequence learning with RNNs

### Listing 11.28 GRU-based encoder

Let’s implement this in Keras with GRU-based encoders and decoders. The choice
of GRU rather than LSTM makes things a bit simpler, since GRU only has a single
state vector, whereas LSTM has multiple. Let’s start with the encoder.

Q: Difference in GRU and LSTM layers

In [24]:
from tensorflow import keras
from tensorflow.keras import layers

embed_dim = 256
latent_dim = 1024

source = keras.Input(shape=(None,), dtype="int64", name="english")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(source)
encoded_source = layers.Bidirectional(
    layers.GRU(latent_dim), merge_mode="sum")(x)

### Listing 11.29 GRU-based decoder and the end-to-end model

During training, the decoder takes as input the entire target sequence, but thanks to
the step-by-step nature of RNNs, it only looks at tokens 0…N in the input to predict token N in the output (which corresponds to the next token in the sequence, since
the output is intended to be offset by one step). This means we only use information
from the past to predict the future, as we should; otherwise we’d be cheating, and our
model would not work at inference time.

In [25]:
past_target = keras.Input(shape=(None,), dtype="int64", name="spanish")

x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(past_target)
decoder_gru = layers.GRU(latent_dim, return_sequences=True)

x = decoder_gru(x, initial_state=encoded_source)
x = layers.Dropout(0.5)(x)
target_next_step = layers.Dense(vocab_size, activation="softmax")(x)

# End-to-end model: maps the source sentence and the target sentence to the target sentence one step in the future
seq2seq_rnn = keras.Model([source, past_target], target_next_step)


### Listing 11.30 Training our recurrent sequence-to-sequence model

In [26]:
seq2seq_rnn.compile(optimizer="rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
seq2seq_rnn.summary()


Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ english (InputLayer)      │ (None, None)           │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ spanish (InputLayer)      │ (None, None)           │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_5 (Embedding)   │ (None, None, 256)      │      3,840,000 │ english[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ not_equal (NotEqual)      │ (None, None)           │              0 │ english[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_6 (Embedding)   │ (None, None, 256)      │      3,840,000 │ spanish[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional             │ (None, 1024)           │      7,876,608 │ embedding_5[0][0],     │
│ (Bidirectional)           │                        │                │ not_equal[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ gru_1 (GRU)               │ (None, None, 1024)     │      3,938,304 │ embedding_6[0][0],     │
│                           │                        │                │ bidirectional[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_6 (Dropout)       │ (None, None, 1024)     │              0 │ gru_1[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_10 (Dense)          │ (None, None, 15000)    │     15,375,000 │ dropout_6[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 34,869,912 (133.02 MB)

 Trainable params: 34,869,912 (133.02 MB)

 Non-trainable params: 0 (0.00 B)

In [27]:
seq2seq_rnn.fit(train_ds, epochs=15, validation_data=val_ds)

Epoch 1/15
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 215s 160ms/step - accuracy: 0.1498 - loss: 5.2701 - val_accuracy: 0.1564 - val_loss: 3.9309
Epoch 2/15
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 208s 160ms/step - accuracy: 0.1597 - loss: 3.8961 - val_accuracy: 0.1891 - val_loss: 3.2533
Epoch 3/15
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 208s 159ms/step - accuracy: 0.1860 - loss: 3.3316 - val_accuracy: 0.2088 - val_loss: 2.8836
Epoch 4/15
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 208s 159ms/step - accuracy: 0.2029 - loss: 2.9563 - val_accuracy: 0.2223 - val_loss: 2.6432
Epoch 5/15
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 208s 159ms/step - accuracy: 0.2171 - loss: 2.6647 - val_accuracy: 0.2339 - val_loss: 2.4469
Epoch 6/15
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 262s 159ms/step - accuracy: 0.2289 - loss: 2.4303 - val_accuracy: 0.2425 - val_loss: 2.3102
Epoch 7/15
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 207s 159ms/step - accuracy: 0.2392 - loss: 2.2369 - val_accuracy: 0.2483 - val_loss: 2.2132
Epoch 8/15
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 207s 159ms/step - ac

### Listing 11.31 Translating new sentences with our RNN encoder and decoder

In [28]:
import numpy as np

#Prepare a dict to convert token
#index predictions to string tokens

spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
  tokenized_input_sentence = source_vectorization([input_sentence])
  decoded_sentence = "[start]"
  for i in range(max_decoded_sentence_length):
    tokenized_target_sentence = target_vectorization([decoded_sentence])
    next_token_predictions = seq2seq_rnn.predict(
        [tokenized_input_sentence, tokenized_target_sentence])
    sampled_token_index = np.argmax(next_token_predictions[0, i, :])
    sampled_token = spa_index_lookup[sampled_token_index]
    decoded_sentence += " " + sampled_token
    if sampled_token == "[end]":
      break
  return decoded_sentence

In [29]:
test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(30):
  input_sentence = random.choice(test_eng_texts)
  print("-")
  print(input_sentence)
  print(decode_sequence(input_sentence))
  print("-"*50)

-
The strong wind indicates that a storm is coming.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 275ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
[start] la fuerte más [UNK] que ha [UNK] es una hora [end]
--------------------------------------------------
-
They caught the robbers red-handed.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
[start] [UNK] la pelota de [UNK] [end]
--------------------------------------------------
-
It is never too late to learn.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34

### Listing 11.33 The TransformerDecoder

In [36]:
class TransformerDecoder(layers.Layer):
  def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
    super().__init__(**kwargs)
    self.embed_dim = embed_dim
    self.dense_dim = dense_dim
    self.num_heads = num_heads
    self.attention_1 = layers.MultiHeadAttention( num_heads=num_heads, key_dim=embed_dim)
    self.attention_2 = layers.MultiHeadAttention( num_heads=num_heads, key_dim=embed_dim)
    self.dense_proj = keras.Sequential([layers.Dense(dense_dim, activation="relu"), layers.Dense(embed_dim),] )
    self.layernorm_1 = layers.LayerNormalization()
    self.layernorm_2 = layers.LayerNormalization()
    self.layernorm_3 = layers.LayerNormalization()
    self.supports_masking = True

  def get_config(self):
    config = super().get_config()
    config.update({
        "embed_dim": self.embed_dim,
        "num_heads": self.num_heads,
        "dense_dim": self.dense_dim,
    })
    return config

 #Generate matrix of shape (sequence_length, sequence_length) with 1s in one half and 0s in the other.
  def get_causal_attention_mask(self, inputs):
    input_shape = tf.shape(inputs)
    batch_size, sequence_length = input_shape[0], input_shape[1]
    i = tf.range(sequence_length)[:, tf.newaxis]
    j = tf.range(sequence_length)
    mask = tf.cast(i >= j, dtype="int32")
    mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
    mult = tf.concat([tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)], axis=0)
    return tf.tile(mask, mult)

  def call(self, inputs, encoder_outputs, mask=None):
    causal_mask = self.get_causal_attention_mask(inputs)
    if mask is not None:
      padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
      padding_mask = tf.minimum(padding_mask, causal_mask)
    else:
      padding_mask = mask
    attention_output_1 = self.attention_1(query=inputs, value=inputs, key=inputs, attention_mask=causal_mask)
    attention_output_1 = self.layernorm_1(inputs + attention_output_1)
    attention_output_2 = self.attention_2(query=attention_output_1, value=encoder_outputs, key=encoder_outputs, attention_mask=padding_mask,)
    attention_output_2 = self.layernorm_2(attention_output_1 + attention_output_2)
    proj_output = self.dense_proj(attention_output_2)
    return self.layernorm_3(attention_output_2 + proj_output)


### Listing 11.34 TransformerDecoder method that generates a causal mask


In [ ]:
"""
#Generate matrix of shape (sequence_length, sequence_length) with 1s in one half and 0s in the other.
def get_causal_attention_mask(self, inputs):
  input_shape = tf.shape(inputs)
  batch_size, sequence_length = input_shape[0], input_shape[1]
  i = tf.range(sequence_length)[:, tf.newaxis]
  j = tf.range(sequence_length)
  mask = tf.cast(i >= j, dtype="int32")

  #Replicate it along the batch axis to get a matrix of shape (batch_size, sequence_length, sequence_length)
  mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
  mult = tf.concat([tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)], axis=0)
  return tf.tile(mask, mult)
"""

### Listing 11.35 The forward pass of the TransformerDecoder

In [ ]:
"""
def call(self, inputs, encoder_outputs, mask=None):
  causal_mask = self.get_causal_attention_mask(inputs)
  if mask is not None:
    padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
    padding_mask = tf.minimum(padding_mask, causal_mask)
  attention_output_1 = self.attention_1(query=inputs, value=inputs, key=inputs, attention_mask=causal_mask)
  attention_output_1 = self.layernorm_1(inputs + attention_output_1)
  attention_output_2 = self.attention_2(query=attention_output_1, value=encoder_outputs, key=encoder_outputs, attention_mask=padding_mask,)
  attention_output_2 = self.layernorm_2(attention_output_1 + attention_output_2)
  proj_output = self.dense_proj(attention_output_2)
  return self.layernorm_3(attention_output_2 + proj_output)
"""

### Listing 11.36 End-to-end Transformer

In [37]:
embed_dim = 256
dense_dim = 2048
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="english")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="spanish")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)
transformer.summary()

Model: "functional_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ english (InputLayer)      │ (None, None)           │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ spanish (InputLayer)      │ (None, None)           │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ positional_embedding_7    │ (None, None, 256)      │      3,845,120 │ english[0][0]          │
│ (PositionalEmbedding)     │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ positional_embedding_8    │ (None, None, 256)      │      3,845,120 │ spanish[0][0]          │
│ (PositionalEmbedding)     │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ transformer_encoder_5     │ (None, None, 256)      │      3,155,456 │ positional_embedding_… │
│ (TransformerEncoder)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ transformer_decoder_3     │ (None, None, 256)      │      5,259,520 │ positional_embedding_… │
│ (TransformerDecoder)      │                        │                │ transformer_encoder_5… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_15 (Dropout)      │ (None, None, 256)      │              0 │ transformer_decoder_3… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_27 (Dense)          │ (None, None, 15000)    │      3,855,000 │ dropout_15[0][0]       │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 19,960,216 (76.14 MB)

 Trainable params: 19,960,216 (76.14 MB)

 Non-trainable params: 0 (0.00 B)

### Listing 11.37 Training the sequence-to-sequence Transformer

In [38]:
transformer.compile(optimizer="rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
transformer.fit(train_ds, epochs=30, validation_data=val_ds)

Epoch 1/30
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 102s 65ms/step - accuracy: 0.7102 - loss: 2.2200 - val_accuracy: 0.7968 - val_loss: 1.2606
Epoch 2/30
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 114s 51ms/step - accuracy: 0.8107 - loss: 1.2119 - val_accuracy: 0.8520 - val_loss: 0.8924
Epoch 3/30
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 67s 52ms/step - accuracy: 0.8505 - loss: 0.9185 - val_accuracy: 0.8660 - val_loss: 0.7824
Epoch 4/30
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 80s 50ms/step - accuracy: 0.8668 - loss: 0.7954 - val_accuracy: 0.8707 - val_loss: 0.7534
Epoch 5/30
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 80s 49ms/step - accuracy: 0.8770 - loss: 0.7214 - val_accuracy: 0.8764 - val_loss: 0.7335
Epoch 6/30
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 63s 48ms/step - accuracy: 0.8844 - loss: 0.6699 - val_accuracy: 0.8792 - val_loss: 0.7201
Epoch 7/30
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 63s 49ms/step - accuracy: 0.8904 - loss: 0.6319 - val_accuracy: 0.8805 - val_loss: 0.7167
Epoch 8/30
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 63s 49ms/step - accuracy: 0.8951

### Listing 11.38 Translating new sentences with our Transformer model

In [39]:
import numpy as np
spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
  tokenized_input_sentence = source_vectorization([input_sentence])
  decoded_sentence = "[start]"
  for i in range(max_decoded_sentence_length):
    tokenized_target_sentence = target_vectorization([decoded_sentence])[:, :-1]
    predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])
    sampled_token_index = np.argmax(predictions[0, i, :])
    sampled_token = spa_index_lookup[sampled_token_index]
    decoded_sentence += " " + sampled_token
    if sampled_token == "[end]":
      break
  return decoded_sentence


In [40]:
test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(20):
  input_sentence = random.choice(test_eng_texts)
  print("-")
  print(input_sentence)
  print(decode_sequence(input_sentence))

-
I am afraid it will be rainy.
[start] me temo que [UNK] estará [UNK] [end]
-
The mouse ran into the hole.
[start] el ratón entró corriendo al agujero [end]
-
I found this under your bed.
[start] encontré esta cama debajo de la cama [end]
-
Come back home.
[start] vuelve a casa [end]
-
In our next class, we will study the days of the week, the months, and the seasons.
[start] en la siguiente clase [UNK] la semana [end]
-
If you're not careful, you might hurt yourself.
[start] si no tienes cuidado [UNK] podría herir [end]
-
These things aren't needed.
[start] éstos no están [UNK] [end]
-
Success breeds success.
[start] el éxito engendra éxito [end]
-
What did he say?
[start] qué dijo él [end]
-
There's something on the table.
[start] hay algo en la mesa [end]
-
Florence is the most beautiful city in Italy.
[start] las fotografías de la ciudad más hermosa italia [end]
-
Don't take the bait.
[start] no [UNK] la [UNK] [end]
-
I wanted you to come.
[start] quería que vengas [end]
-
Tom see